In [ ]:
# 📢 Conversational Interface for NewsBot 2.0
import re
from collections import defaultdict
from typing import Dict, Any, List

class ConversationalInterface:
    """
    Section 5 implementation:
    - Intent classification
    - Entity extraction
    - Context management
    - Response generation
    - Follow-up handling
    """

    def __init__(self, newsbot_adapter):
        """
        newsbot_adapter: an object with methods:
          search(), summarize(), analyze(), compare(), explain()
        """
        self.newsbot = newsbot_adapter
        self.context = {}  # store last_intent, last_entities

        # Intent keywords (you can expand as needed)
        self.intent_keywords = {
            "search": ["show", "find", "news", "search", "get", "list", "give me"],
            "summarize": ["summarize", "summary", "brief", "short"],
            "analyze": ["analyze", "analysis", "sentiment", "trend", "trends"],
            "compare": ["compare", "versus", "vs", "difference", "contrast"],
            "explain": ["explain", "what is", "how", "relationship", "relations"]
        }

    # Intent classification
    def classify_intent(self, user_query: str) -> str:
        q = user_query.lower()
        for intent in ["compare", "analyze", "summarize", "explain", "search"]:
            for kw in self.intent_keywords[intent]:
                if kw in q:
                    return intent
        return "search"

    # Entity extraction
    def extract_query_entities(self, user_query: str) -> Dict[str, Any]:
        # If spaCy is available in your Midterm code, use it
        try:
            doc = nlp(user_query)
            entities = defaultdict(list)
            for ent in doc.ents:
                entities[ent.label_].append(ent.text)
        except NameError:
            entities = defaultdict(list)

        # Categories
        categories = ["tech", "technology", "sports", "economy", "business", "politics", "health"]
        for cat in categories:
            if re.search(r"\b" + re.escape(cat) + r"\b", user_query, flags=re.I):
                entities["category"].append(cat)

        # Timeframe extraction
        if re.search(r"\bthis month\b|\blast month\b", user_query, flags=re.I):
            entities["timeframe"].append("month")
        if re.search(r"\bthis week\b|\blast week\b", user_query, flags=re.I):
            entities["timeframe"].append("week")
        if re.search(r"\btoday\b|\bthis morning\b|\bthis afternoon\b", user_query, flags=re.I):
            entities["timeframe"].append("today")

        # Sentiment
        if re.search(r"\bpositive\b|\bnegative\b|\bneutral\b", user_query, flags=re.I):
            match = re.search(r"\b(positive|negative|neutral)\b", user_query, flags=re.I)
            if match:
                entities["sentiment"].append(match.group(1).lower())

        # Company names (fallback if spaCy not available)
        if "companies" not in entities or not entities["companies"]:
            titles = re.findall(r"\b[A-Z][a-z]{2,}(?:\s[A-Z][a-z]{2,})*\b", user_query)
            filtered = [t for t in titles if t.lower() not in {"this", "please", "what", "who", "show"}]
            if filtered:
                entities["companies"].extend(filtered)

        # Flatten single values
        out = {}
        for k, v in entities.items():
            vals = list(dict.fromkeys(v))
            out[k.lower()] = vals if len(vals) > 1 else (vals[0] if vals else [])
        return out

    # Execute intent
    def execute(self, intent: str, entities: Dict[str, Any]) -> List[str]:
        if intent == "search":
            return self.newsbot.search(entities)
        elif intent == "summarize":
            return self.newsbot.summarize(entities)
        elif intent == "analyze":
            return self.newsbot.analyze(entities)
        elif intent == "compare":
            return self.newsbot.compare(entities)
        elif intent == "explain":
            return self.newsbot.explain(entities)
        else:
            return ["I couldn't understand the action you want."]

    # Format response
    def generate_response(self, results: List[str], intent: str, entities: Dict[str, Any]) -> str:
        ent_summary = ", ".join(f"{k}={v}" for k, v in entities.items() if v)
        header = {
            "search": f"🔍 Search results {f'({ent_summary})' if ent_summary else ''}:",
            "summarize": f"📝 Summary {f'({ent_summary})' if ent_summary else ''}:",
            "analyze": f"📊 Analysis {f'({ent_summary})' if ent_summary else ''}:",
            "compare": f"⚖️ Comparison {f'({ent_summary})' if ent_summary else ''}:",
            "explain": f"🧠 Explanation {f'({ent_summary})' if ent_summary else ''}:",
        }.get(intent, "Results:")
        body = "\n".join(f"{i+1}. {r}" for i, r in enumerate(results))
        return f"{header}\n{body}"

    # Process new query
    def process_query(self, user_query: str) -> str:
        intent = self.classify_intent(user_query)
        entities = self.extract_query_entities(user_query)
        self.context["last_intent"] = intent
        self.context["last_entities"] = entities
        results = self.execute(intent, entities)
        return self.generate_response(results, intent, entities)

    # Handle follow-up query
    def handle_follow_up(self, follow_up_query: str) -> str:
        new_entities = self.extract_query_entities(follow_up_query)
        merged = dict(self.context.get("last_entities", {}))
        for k, v in new_entities.items():
            if v:
                merged[k] = v
        intent = self.context.get("last_intent", self.classify_intent(follow_up_query))
        self.context["last_entities"] = merged
        self.context["last_intent"] = intent
        results = self.execute(intent, merged)
        return self.generate_response(results, intent, merged)


# Example usage with a dummy adapter for testing
class DummyNewsBotAdapter:
    def search(self, entities): return [f"Found news for {entities}"]
    def summarize(self, entities): return [f"Summary for {entities}"]
    def analyze(self, entities): return [f"Analysis for {entities}"]
    def compare(self, entities): return [f"Comparison for {entities}"]
    def explain(self, entities): return [f"Explanation for {entities}"]

if __name__ == "__main__":
    adapter = DummyNewsBotAdapter()
    ci = ConversationalInterface(adapter)

    print(ci.process_query("Show me positive tech news from this week"))
    print(ci.handle_follow_up("What about last month?"))
